In [3]:
#!pip install pyOpenBCI pylsl
#!pip install numpy pyserial bitstring xmltodict requests

In [2]:
# import pyOpenBCI
# from pyOpenBCI import OpenBCICyton

# #For Windows replace '*' with the port number
# #board = OpenBCICyton(port='COM*')

# #For MacOS and Linux replace '*' with the port number
# #board = OpenBCICyton(port='/dev/ttyUSB*')

# #Auto port detection
# board = OpenBCICyton()

In [1]:
# from pyOpenBCI import OpenBCICyton

# def print_raw(sample):
#     print(sample.channels_data)

# board = OpenBCICyton()

# board.start_stream(print_raw)

In [16]:
#Write commands to the board
#board.write_command(command)

import pyOpenBCI
from pyOpenBCI import OpenBCICyton
import numpy as np

#Commands
def print_raw(sample):
    print(sample.channels_data, sample.aux_data)

def return_raw(sample):
    return sample.channels_data, sample.aux_data

def return_filtered(sample):
    SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
    SCALE_FACTOR_AUX = 0.002 / (2**4)

    return np.array(sample.channels_data) * SCALE_FACTOR_EEG, np.array(sample.aux_data) * SCALE_FACTOR_AUX

def record_data(sample, initial):
    SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
    SCALE_FACTOR_AUX = 0.002 / (2**4)
    #open a csv file named (initial, _eeg) and write the data to it
    with open(initial + '_eeg.csv', 'a') as file:
        file.write(sample.channels_data * SCALE_FACTOR_EEG, sample.aux_data * SCALE_FACTOR_AUX)

def rec_data_labeled(sample, initial, labels):
    SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
    SCALE_FACTOR_AUX = 0.002 / (2**4)
    labl = []
    labl.append(labels)
    #open a csv file named (initial, _eeg) and write the data to it
    with open(initial + '_eeg.csv', 'a') as file:
        file.write(np.array(sample.channels_data) * SCALE_FACTOR_EEG, np.array(sample.aux_data) * SCALE_FACTOR_AUX, labl)

def stream_data(comm):
    board = pyOpenBCI.OpenBCICyton()
    board.start_stream(comm)
    return board

#Test use
initial = 'test'
labels = 'test'

board = pyOpenBCI.OpenBCICyton()
board.start_stream(return_filtered)
#board = stream_data(rec_data_labeled(board, initial, labels))

Serial established
Skipped 100 bytes before start found


KeyboardInterrupt: 

In [1]:
#Lets create a tkinter window that connects the headset and displays the data
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import pyOpenBCI
from pylsl import StreamInfo, StreamOutlet
import numpy as np

#Create a window
win = tk.Tk()
win.title("OpenBCI GUI")

#Label CogniSync
pgLabel = ttk.Label(win, text="CogniSync")
pgLabel.grid(row=0, column=0)

#Button for connecting to the headset
def startLSL():
    #messagebox.showinfo("Connection", "Connecting to the headset")
    SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
    SCALE_FACTOR_AUX = 0.002 / (2**4)
    print("Creating LSL stream for EEG. \nName: OpenBCIEEG\nID: OpenBCItestEEG\n")
    info_eeg = StreamInfo('OpenBCIEEG', 'EEG', 8, 250, 'float32', 'OpenBCItestEEG')
    print("Creating LSL stream for AUX. \nName: OpenBCIAUX\nID: OpenBCItestEEG\n")
    info_aux = StreamInfo('OpenBCIAUX', 'AUX', 3, 250, 'float32', 'OpenBCItestAUX')

    outlet_eeg = StreamOutlet(info_eeg)
    outlet_aux = StreamOutlet(info_aux)

    def lsl_streamers(sample):
        outlet_eeg.push_sample(np.array(sample.channels_data)*SCALE_FACTOR_EEG)
        outlet_aux.push_sample(np.array(sample.aux_data)*SCALE_FACTOR_AUX)

    board = pyOpenBCI.OpenBCICyton()
    board.start_stream(lsl_streamers)

#Function to retreive the data from our lsl stream
def getData():
    print("Retreiving data from the LSL stream")
    

#Create a button
connectButton = ttk.Button(win, text="Connect", command=startLSL)
connectButton.grid(row=1, column=1)


#Display the window
win.mainloop()

Creating LSL stream for EEG. 
Name: OpenBCIEEG
ID: OpenBCItestEEG

Creating LSL stream for AUX. 
Name: OpenBCIAUX
ID: OpenBCItestEEG



2024-02-05 14:22:21.774 (  24.240s) [          160995]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2024-02-05 14:22:21.776 (  24.241s) [          160995]      netinterfaces.cpp:91    I

Serial established
Skipped 100 bytes before start found


: 

In [43]:
from pyOpenBCI import OpenBCICyton
from pylsl import StreamInfo, StreamOutlet
import numpy as np

SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
SCALE_FACTOR_AUX = 0.002 / (2**4)

print("Creating LSL stream for EEG. \nName: OpenBCIEEG\nID: OpenBCItestEEG\n")

info_eeg = StreamInfo('OpenBCIEEG', 'EEG', 8, 250, 'float32', 'OpenBCItestEEG')

print("Creating LSL stream for AUX. \nName: OpenBCIAUX\nID: OpenBCItestEEG\n")

info_aux = StreamInfo('OpenBCIAUX', 'AUX', 3, 250, 'float32', 'OpenBCItestAUX')

outlet_eeg = StreamOutlet(info_eeg)
outlet_aux = StreamOutlet(info_aux)
file = open('test.txt', 'a')
hi = open('hi.txt', 'a')
def lsl_streamers(sample):
    lbl = ''
    if hi == None:
        lbl = 'null'
    else:
        lbl = hi.read()
    outlet_eeg.push_sample(np.array(sample.channels_data)*SCALE_FACTOR_EEG)
    outlet_aux.push_sample(np.array(sample.aux_data)*SCALE_FACTOR_AUX)
    print(sample.channels_data)
    file.write(str(np.array(sample.channels_data)) + str(np.array(sample.aux_data)) + str(lbl) + '\n')

board = OpenBCICyton()

board.start_stream(lsl_streamers)
file.close()

Creating LSL stream for EEG. 
Name: OpenBCIEEG
ID: OpenBCItestEEG

Creating LSL stream for AUX. 
Name: OpenBCIAUX
ID: OpenBCItestEEG



2024-02-05 21:07:54.055 (2819.291s) [          1653B8]         udp_server.cpp:82    WARN| Could not bind multicast responder for ff02:113d:6fdd:2c17:a643:ffe2:1bd1:3cd2 to interface ::1 (Can't assign requested address)
2024-02-05 21:07:54.055 (2819.291s) [          1653B8]         udp_server.cpp:82    WARN| Could not bind multicast responder for ff02:113d:6fdd:2c17:a643:ffe2:1bd1:3cd2 to interface fdad:9943:e9c2:868:1cd8:8006:8b39:371c (Can't assign requested address)
2024-02-05 21:07:54.055 (2819.291s) [          1653B8]         udp_server.cpp:82    WARN| Could not bind multicast responder for ff02:113d:6fdd:2c17:a643:ffe2:1bd1:3cd2 to interface 2607:fb91:79f3:e2ca:e4f2:b6d7:d081:c5ca (Can't assign requested address)
2024-02-05 21:07:54.055 (2819.291s) [          1653B8]         udp_server.cpp:82    WARN| Could not bind multicast responder for ff05:113d:6fdd:2c17:a643:ffe2:1bd1:3cd2 to interface ::1 (Can't assign requested address)
2024-02-05 21:07:54.055 (2819.291s) [          1653B8

Serial established
Skipped 100 bytes before start found
[0, 0, 0, 0, 0, 0, 0, 0]
[-5122192, -5301929, -5317022, 7268199, -5050216, 8388607, -4807598, -4343272]
[-5262617, -5285994, -5276509, 7277415, -5055626, 8388607, -4627463, -4084864]
[-5293972, -5093868, -5069306, 7293744, -5011874, 8388607, -4409458, -3918420]
[-5167046, -5109681, -5169943, 7287271, -5002531, 8388607, -4556387, -4145646]
[-5102222, -5289317, -5312172, 7269514, -5042635, 8388607, -4812819, -4352436]
[-5235910, -5299828, -5296262, 7274208, -5059646, 8388607, -4695729, -4142961]
[-5299164, -5128711, -5098431, 7292163, -5018949, 8388607, -4441643, -3919132]
[-5189000, -5076987, -5128329, 7290212, -4999691, 8388607, -4523760, -4090796]
[-5089536, -5270194, -5303228, 7271611, -5034478, 8388607, -4803894, -4346084]
[-5204309, -5307616, -5308256, 7271329, -5061611, 8388607, -4758962, -4200723]
[-5298922, -5167371, -5136219, 7289733, -5026676, 8388607, -4485464, -3933662]
[-5211574, -5054540, -5091069, 7292426, -4998511, 

KeyboardInterrupt: 